In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#Loading the dataset
dataset = pd.read_csv("input/ks-projects-201801.csv")
dataset=dataset[(dataset['state'] == 'failed')|(dataset['state']=='canceled') | (dataset['state'] == 'successful')]
print(dataset.shape)
dataset.drop(dataset.index[dataset['country'] == 'N,0"'], inplace = True)
print(dataset.shape)
dataset.drop(dataset.index[dataset['launched'] == '1970-01-01 01:00:00'], inplace = True)
print(dataset.shape)

onehotcurrency = pd.get_dummies(dataset['currency'])
dataset = onehotcurrency.join(dataset)
#dataset.shape

onehotcountry = pd.get_dummies(dataset['country'])
dataset = onehotcountry.join(dataset)
#dataset.shape

onehotmaincategory = pd.get_dummies(dataset['main_category'])
dataset = onehotmaincategory.join(dataset)
#dataset.shape

onehotstate = pd.get_dummies(dataset['state'])
dataset = onehotstate.join(dataset)
#dataset.shape

dataset['launched'] = pd.to_datetime(dataset['launched'])
dataset['laun_month_year'] = dataset['launched'].dt.to_period("M")
dataset['laun_day_month_year'] = dataset['launched'].dt.to_period("D")
dataset['laun_year'] = dataset['launched'].dt.to_period("A")
dataset['laun_hour'] = dataset['launched'].dt.hour

dataset['deadline'] = pd.to_datetime(dataset['deadline'])
dataset['dead_month_year'] = dataset['deadline'].dt.to_period("M")
dataset['dead_day_month_year'] = dataset['deadline'].dt.to_period("D")
dataset['dead_year'] = dataset['deadline'].dt.to_period("A")


#Creating a new columns with Campaign total months
#dataset['time_campaign'] = dataset['dead_month_year'] - dataset['laun_month_year']
dataset['time_campaign'] = (((dataset.dead_day_month_year - dataset.laun_day_month_year)/np.timedelta64(1, 'M'))).astype(int)
dataset.shape

(370454, 15)
(370222, 15)
(370216, 15)


(370216, 77)

In [15]:
dataset.info()
dataset.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 370216 entries, 0 to 378660
Data columns (total 77 columns):
canceled               370216 non-null uint8
failed                 370216 non-null uint8
successful             370216 non-null uint8
Art                    370216 non-null uint8
Comics                 370216 non-null uint8
Crafts                 370216 non-null uint8
Dance                  370216 non-null uint8
Design                 370216 non-null uint8
Fashion                370216 non-null uint8
Film & Video           370216 non-null uint8
Food                   370216 non-null uint8
Games                  370216 non-null uint8
Journalism             370216 non-null uint8
Music                  370216 non-null uint8
Photography            370216 non-null uint8
Publishing             370216 non-null uint8
Technology             370216 non-null uint8
Theater                370216 non-null uint8
AT                     370216 non-null uint8
AU                     370216 non-

,canceled,failed,successful,Art,Comics,Crafts,Dance,Design,Fashion,Film & Video,...,usd_pledged_real,usd_goal_real,laun_month_year,laun_day_month_year,laun_year,laun_hour,dead_month_year,dead_day_month_year,dead_year,time_campaign
0,0,1,0,0,0,0,0,0,0,0,...,0.0,1533.95,2015-08,2015-08-11,2015,12,2015-10,2015-10-09,2015,<2 * MonthEnds>
1,0,1,0,0,0,0,0,0,0,1,...,2421.0,30000.00,2017-09,2017-09-02,2017,4,2017-11,2017-11-01,2017,<2 * MonthEnds>
2,0,1,0,0,0,0,0,0,0,1,...,220.0,45000.00,2013-01,2013-01-12,2013,0,2013-02,2013-02-26,2013,<MonthEnd>
3,0,1,0,0,0,0,0,0,0,0,...,1.0,5000.00,2012-03,2012-03-17,2012,3,2012-04,2012-04-16,2012,<MonthEnd>
4,1,0,0,0,0,0,0,0,0,1,...,1283.0,19500.00,2015-07,2015-07-04,2015,8,2015-08,2015-08-29,2015,<MonthEnd>


In [16]:
print(dataset.nunique())

canceled                    2
failed                      2
successful                  2
Art                         2
Comics                      2
Crafts                      2
Dance                       2
Design                      2
Fashion                     2
Film & Video                2
Food                        2
Games                       2
Journalism                  2
Music                       2
Photography                 2
Publishing                  2
Technology                  2
Theater                     2
AT                          2
AU                          2
BE                          2
CA                          2
CH                          2
DE                          2
DK                          2
ES                          2
FR                          2
GB                          2
HK                          2
IE                          2
                        ...  
JPY                         2
MXN                         2
NOK       

In [ ]:
X= pd.concat([dataset.ix[:,3:55],dataset.ix[:,64:65], dataset.ix[:,67:69], dataset.ix[:,76:]], axis=1)
y = dataset.iloc[:,63]

#print(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.25,random_state=0)

from sklearn.preprocessing import StandardScaler
sc_X= StandardScaler()
X_train= sc_X.fit_transform(X_train)
X_test= sc_X.transform(X_test)


In [ ]:
#Backward Elimination
import statsmodels.formula.api as sm 

df_feat[[c for c in df_feat.columns if c != "name"]].head()
X= pd.concat([dataset.ix[:,3:55],dataset.ix[:,64:65], dataset.ix[:,67:69], dataset.ix[:,76:]], axis=1)
y = dataset.iloc[:,63]
#features = [c for c in X.columns if c not in ["state", "name", "ID","category","main_category","goal","launched","country","usd pledged,"pledged","usd_pledged_real","diff_pledged_goal_real"]]
#X = X[features] # choosing initial features

cols = list(X.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X = X[cols]
    X_1 = np.append(arr=np.ones((X.shape[0],1)).astype(int), values=X, axis=1) #Adding constant column of ones, mandatory for sm.OLS model
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)
 

### Logistic Regression

In [22]:
from sklearn.linear_model import LogisticRegression
classifierObj= LogisticRegression(random_state=0)
classifierObj.fit(X_train, y_train)

#Making predictions on the Test Set
y_pred= classifierObj.predict(X_test)

#Evaluating the predictions using a Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

classifierObj.score(X_test,y_test)

/home/hmnshu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/hmnshu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7813384618709078

### KNN

In [ ]:
#Fitting Classifier to Training Set. Create a classifier object here and call it classifierObj
from sklearn.neighbors import KNeighborsClassifier
classifierObj= KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
classifierObj.fit(X_train, y_train) 

#Making predictions on the Test Set
y_pred= classifierObj.predict(X_test)

#Evaluating the predictions using a Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
classifierObj.score(X_test,y_test)

### SVM

In [ ]:
#Fitting Classifier to Training Set. Create a classifier object here and call it classifierObj
#Default kernel it uses is RBF, which is a gaussian kernel. For now we will specify that we want to use a “linear” kernel. 
from sklearn.svm import SVC
classifierObj= SVC(kernel='linear')
classifierObj.fit(X_train, y_train) 

#Making predictions on the Test Set
y_pred= classifierObj.predict(X_test)

#Evaluating the predictions using a Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
classifierObj.score(X_test,y_test)

#from sklearn.model_selection import cross_val_score
#modelAccuracies= cross_val_score(estimator=classifierObj, X=X_train, y=y_train, cv=10)
#print(modelAccuracies.mean())
#print(modelAccuracies.std())

### Kernel SVM

In [ ]:
#Fitting Classifier to Training Set. Create a classifier object here and call it classifierObj
from sklearn.svm import SVC
classifierObj= SVC()
#try different degrees of polynomial kernels
#classifierObj= SVC(kernel='poly', degree=3)
#classifierObj=SVC(kernel='sigmoid')
classifierObj.fit(X_train, y_train)

#Making predictions on the Test Set
y_pred= classifierObj.predict(X_test)

#Evaluating the predictions using a Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
classifierObj.score(X_test,y_test)

### Naive Bayes

In [ ]:
#Fitting Classifier to Training Set. Create a classifier object here and call it classifierObj
from sklearn.naive_bayes import GaussianNB
classifierObj= GaussianNB()
classifierObj.fit(X_train, y_train) 

#Making predictions on the Test Set
y_pred= classifierObj.predict(X_test)

#Evaluating the predictions using a Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
classifierObj.score(X_test,y_test)

### Decision Tree

In [ ]:
#Fitting Classifier to Training Set. Create a classifier object here and call it classifierObj
from sklearn.tree import DecisionTreeClassifier
classifierObj= DecisionTreeClassifier(criterion='entropy')
classifierObj.fit(X_train,y_train)

#Making predictions on the Test Set
y_pred= classifierObj.predict(X_test)

#Evaluating the predictions using a Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
classifierObj.score(X_test,y_test)

### Random Forest

In [23]:
#Fitting Classifier to Training Set. Create a classifier object here and call it classifierObj
from sklearn.ensemble import RandomForestClassifier
classifierObj= RandomForestClassifier(criterion='entropy')
classifierObj.fit(X_train,y_train)

#Making predictions on the Test Set
y_pred= classifierObj.predict(X_test)

#Evaluating the predictions using a Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
classifierObj.score(X_test,y_test)

/home/hmnshu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8607839747606802

In [7]:
#Applying LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
ldaObj= LDA(n_components=55)
X_train= ldaObj.fit_transform(X_train,y_train)
X_test= ldaObj.transform(X_test)

C:\Users\jwang2\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [8]:
#Fitting Classifier to Training Set. Create a classifier object here and call it classifierObj
from sklearn.ensemble import RandomForestClassifier
classifierObj= RandomForestClassifier(criterion='entropy')
classifierObj.fit(X_train,y_train)

#Making predictions on the Test Set
y_pred= classifierObj.predict(X_test)

#Evaluating the predictions using a Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
classifierObj.score(X_test,y_test)

C:\Users\jwang2\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.713816500280911

In [9]:
from sklearn.linear_model import LogisticRegression
classifierObj= LogisticRegression(random_state=0)
classifierObj.fit(X_train, y_train)

#Making predictions on the Test Set
y_pred= classifierObj.predict(X_test)

#Evaluating the predictions using a Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

classifierObj.score(X_test,y_test)

C:\Users\jwang2\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jwang2\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.571394615151908

In [ ]:
#Applying PCA
from sklearn.decomposition import PCA
pcaObj= PCA(n_components=None)
X_train= pcaObj.fit_transform(X_train)
X_test= pcaObj.transform(X_test)
components_variance= pcaObj.explained_variance_ratio_